# Data Cleaning

## Text Extraction

In [1]:
%pip install pandas requests beautifulsoup4 openpyxl

Note: you may need to restart the kernel to use updated packages.


In [5]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import time
import re

df = pd.read_excel('web_scraping_urls.xlsx', sheet_name='2020 - 2024')

def clean_text(text):
    return re.sub(r'[\x00-\x1f\x7f-\x9f]', '', text)

def extract_text(url):
    try:
        response = requests.get(url)
        response.raise_for_status()
        soup = BeautifulSoup(response.text, 'html.parser')
        return clean_text(soup.get_text())
    except requests.RequestException as e:
        return str(e)

batch_size = 10
output_filename = 'all_extracted_texts.xlsx'

with pd.ExcelWriter(output_filename, engine='openpyxl', mode='a', if_sheet_exists='overlay') as writer:
    total_urls = len(df['URLs'])
    for start in range(0, total_urls, batch_size):
        end = start + batch_size
        print(f"processing batch from URL {start+1} to {min(end, total_urls)}")
        batch_data = {'URL': df['URLs'][start:end], 'Quarter': df['Quarter'][start:end], 'Extracted Text': []}

        for index, url in enumerate(df['URLs'][start:end], start=start+1):
            text = extract_text(url)
            batch_data['Extracted Text'].append(text)
            print(f"extracted from URL {index}: {text[:60]}...")
            time.sleep(2)

        batch_df = pd.DataFrame(batch_data)
        print(batch_df.head())  # Print head of the DataFrame to check it
        batch_df.to_excel(writer, sheet_name='2020 - 2024', index=False, header=not writer.sheets, startrow=writer.sheets['2020 - 2024'].max_row if '2020 - 2024' in writer.sheets else 0)

        print(f"batch from URL {start+1} to {min(end, total_urls)} saved.")


processing batch from URL 1 to 10
extracted from URL 1: 403 Client Error: Forbidden for url: https://www.colliers.co...
extracted from URL 2: Singapore industrial: Strong growth prospects but no easy ri...
extracted from URL 3: 403 Client Error: Forbidden for url: https://www.colliers.co...
extracted from URL 4: Moderation in industrial rent and price growth in 4Q23Skip t...
extracted from URL 5: Industrial rents, prices rose for 13th straight quarter in 4...
extracted from URL 6: 403 Client Error: Forbidden for url: https://sbr.com.sg/comm...
extracted from URL 7: Savills Singapore | Industrial rental growth rate between 0%...
extracted from URL 8: Industrial and Logistics Market Updates Q4 2023 - Knight Fra...
extracted from URL 9: Singapore industrial rental index hits highest annual growth...
extracted from URL 10: 403 Client Error: Forbidden for url: https://www.cbre.com.sg...
                                                 URL  Quarter  \
0  https://www.colliers.com/en-sg/news/2

In [50]:
# initialize the ExcelWriter to append without overwriting previous data
with pd.ExcelWriter(output_filename, engine='openpyxl', mode='a', if_sheet_exists='overlay') as writer:
    total_urls = len(df['URLs'])

    start = 2750
    end = 2757
    batch_data = {'URL': df['URLs'][start:end], 'Quarter': df['Quarter'][start:end], 'Extracted Text': []}

    for index, url in enumerate(df['URLs'][start:end], start=start+1):
        print(f"processing URL {index}/{total_urls}: {url}")
        text = extract_text(url)
        batch_data['Extracted Text'].append(text)
        time.sleep(2)  # sleep for 2 seconds between requests
        print("extraction successful for this URL")

    # convert batch data to DataFrame and append to the Excel file
    batch_df = pd.DataFrame(batch_data)
    # append batch data to the Excel file
    batch_df.to_excel(writer, sheet_name='Extracted Text', index=False, header=not writer.sheets, startrow=writer.sheets['Extracted Text'].max_row if 'Extracted Text' in writer.sheets else 0)

    print(f"batch from URL {start+1} to {min(end, total_urls)} saved.")

print("all URLs processed and saved to the same Excel file.")


ParseError: reference to invalid character number: line 1, column 8659145 (<string>)